In [1]:
import os
import sys
sys.path.append('../src')

%load_ext autoreload
%autoreload 2

### Test Different Modules

Input

In [2]:
PATIENT_LIST = [
    775,  787,  788, 1123, 1169, 1170, 1171, 1172, 1173, 1983, 2110, 2195,
    2955, 2956, 2957, 2958, 2959, 2960, 2961, 2962, 2963, 3081, 3229, 3318, 3432
]

Settings

In [3]:
clinical_score_path = "../data/"
protocol_csv_path = "../data/"

In [4]:
max_values = {
    'BARTHEL': 100,
    'ASH_PROXIMAL': 4,
    'MA_DISTAL': 4,
    'FATIGUE': 63,
    'VAS': 10,
    'FM_A': 36,
    'FM_B': 10,
    'FM_C': 14,
    'FM_D': 6,
    'FM_TOTAL': 66,
    'ACT_AU': 10,
    'ACT_QOM': 10
}

latent_to_clinical_mapping_nest = {
    # Functional Independence
    "BARTHEL": ["DAILY_LIVING_ACTIVITY"],  # Barthel Index measures independence in ADLs.

    # Motor Function (Spasticity & Strength)
    "ASH_PROXIMAL": ["BODY_PART_ARM", "BODY_PART_SHOULDER", "COORDINATION"],  # Ashworth scale for proximal limb spasticity.
    "MA_DISTAL": ["BODY_PART_FINGER", "BODY_PART_WRIST", "GRASPING", "PINCHING"],  # Motor Assessment for distal motor function.

    # Fatigue & Pain
    "FATIGUE": ["DIFFICULTY_COGNITIVE", "DIFFICULTY_MOTOR", "PROCESSING_SPEED", "ATTENTION"],  # Fatigue relates to cognitive/motor difficulty.
    "VAS": ["DIFFICULTY_COGNITIVE", "DIFFICULTY_MOTOR"],  # Visual Analog Scale (VAS) for perceived effort.

    # Fugl-Meyer Subscales (Motor Control & Coordination)
    "FM_A": ["BODY_PART_ARM", "BODY_PART_SHOULDER", "RANGE_OF_MOTION_H", "RANGE_OF_MOTION_V"],  # Upper Limb Motor
    "FM_B": ["BODY_PART_WRIST", "PRONATION_SUPINATION", "RANGE_OF_MOTION_H"],  # Wrist Motor
    "FM_C": ["BODY_PART_FINGER", "GRASPING", "PINCHING"],  # Hand Motor
    "FM_D": ["COORDINATION", "RANGE_OF_MOTION_H", "RANGE_OF_MOTION_V"],  # Coordination & Speed
    "FM_TOTAL": ["BODY_PART_ARM", "BODY_PART_WRIST", "BODY_PART_FINGER", "COORDINATION"],  # Full Upper Limb Score

    # Activity & Movement Quality
    "ACT_AU": ["BODY_PART_TRUNK"],  # Activity Autonomy linked to balance.
    "ACT_QOM": ["COORDINATION"],  # Quality of Movement related to balance & coordination.
}

### Pipeline

Initialize Pipelinne

In [46]:
from ai_cdss.services.pipeline import PipelineBase

In [47]:
pipeline = PipelineBase(
    patient_list = PATIENT_LIST,
    clinical_score_path=clinical_score_path, 
    protocol_csv_path= protocol_csv_path, 
    mapping_dict = latent_to_clinical_mapping_nest, 
    max_subscales = max_values
)

Load Data 

In [48]:
pipeline.load_data()

Database engine created successfully
Data successfully saved to rgs_interaction.csv
Database engine closed


In [206]:
pipeline

In [ ]:
n = 10
top_n_protocols = (
    pipeline.prescriptions.groupby('PATIENT_ID')
    .apply(lambda x: x.nlargest(n, 'Score'), include_groups=True)
    .reset_index(drop=True)
)
top_n_protocols


In [ ]:
# Find top n protocols by score per patient_id
scores.groupby("PATIENT_ID").apply(lambda x: x.nlargest(1, "Score"))

In [ ]:
pipeline.data_processor.get_protocol(protocol_path="../../data/protocol_attributes.csv")

In [ ]:
from ai_cdss.services.prescription import PrescriptionRecommender
pipeline.prescription_recommender = PrescriptionRecommender()

In [ ]:
pipeline.scores

In [ ]:
rank = pipeline.prescription_recommender.rank_protocols(pipeline.prescriptions)

In [ ]:
schedule.columns

In [ ]:
result = pipeline.prescription_recommender.recommend_protocols(pipeline.prescriptions)

In [ ]:
result

In [ ]:
grouped = result.groupby("PATIENT_ID").apply(lambda g: g.to_dict(orient="records")).reset_index(name="PROTOCOLS")


In [ ]:
import msgspec
import pandas as pd
from typing import List

class ProtocolRecommendation(msgspec.Struct):
    PATIENT_ID: int
    PROTOCOL_ID: int
    PPF: float
    ADHERENCE_EWMA: float
    PARAMETER_VALUE_EWMA: float
    PERFORMANCE_VALUE_EWMA: float
    CONTRIBUTION: float
    Score: float
    DAYS: List[int]

class PrescriptionRecommendation(msgspec.Struct):
    PATIENT_ID: int
    PROTOCOLS: List[ProtocolRecommendation]

    @classmethod
    def from_df(cls, df: pd.DataFrame) -> List["PrescriptionRecommendation"]:
        """
        Convert a long-format DataFrame into a list of PrescriptionRecommendation objects.
        """
        grouped = df.groupby("PATIENT_ID").apply(lambda g: g.to_dict(orient="records")).reset_index(name="PROTOCOLS")

        return [
            cls(
                PATIENT_ID=row.PATIENT_ID,
                PROTOCOLS=[ProtocolRecommendation(**p) for p in row.PROTOCOLS]
            )
            for _, row in grouped.iterrows()
        ]

# Apply conversion
prescription_recommendations = PrescriptionRecommendation.from_df(result)

In [ ]:
prescriptions = pipeline.prescriptions
patient_profile = pipeline.patient_profiles

In [ ]:
patient_profile

In [ ]:
import pandas as pd

# Define the real max values based on provided scales
max_values = {
    'BARTHEL': 100,
    'ASH_PROXIMAL': 4,
    'MA_DISTAL': 4,
    'FATIGUE': 63,
    'VAS': 10,
    'FM_A': 36,
    'FM_B': 10,
    'FM_C': 14,
    'FM_D': 6,
    'FM_TOTAL': 66,
    'ACT_AU': 10,
    'ACT_QOM': 10
}

# Compute the deficit matrix: (1 - normalized score)
deficit_matrix = 1 - (patient_profile / pd.Series(max_values))

In [ ]:
deficit_matrix

In [ ]:
feature_names = list(pipeline.protocol_profiles.columns)

In [ ]:
import numpy as np
# import matplotlib.pyplot as plt

# Initialize total_contr as a zero array with the same shape as the contribution lists
total_contr = np.zeros(12)

# Iterate over each row in the DataFrame and sum the "CONTRIBUTION" column
for i, row in p.iterrows():
    contr = np.asarray(row["CONTRIBUTION"])  # Convert the list to a NumPy array
    total_contr += contr  # Element-wise summation

total_contr  # This is now correctly summed

# Normalize values so they sum to 1
total_contr /= np.sum(total_contr)


In [ ]:
feature_names

In [ ]:
total_contr

In [ ]:
top_n_protocols

In [ ]:
top_n_protocols[["PATIENT_ID", "PROTOCOL_ID"]].pivot(index="PATIENT_ID", columns="PROTOCOL_ID", values="PROTOCOL_ID")

In [ ]:
[{int(patient): top_n_protocols[top_n_protocols.PATIENT_ID == patient].PROTOCOL_ID.values} for patient in top_n_protocols.PATIENT_ID.unique()]

In [ ]:
import pandas as pd
import numpy as np

# Create the dataframe from the provided image description
data = {
    "PATIENT_ID": [775, 775, 775, 775, 775, 787, 787, 787, 787, 787],
    "PROTOCOL_ID": [222, 206, 224, 208, 214, 221, 223, 224, 211, 217],
}

df = pd.DataFrame(data)

# Define parameters
n = 5  # protocols per day
m = 7  # number of days

### MultiKeyDict

In [219]:
from ai_cdss.services.data import MultiKeyDict

mkd = MultiKeyDict()

In [221]:
mkd['sub_a'] = ['attr_a', 'attr_b']

In [223]:
mkd.to_yaml('fkfk.yaml')

In [234]:
test = {'sub_a': ['attr_a', 'attr_b']}

In [235]:
test

{'sub_a': ['attr_a', 'attr_b']}

In [225]:
mkd.get('sub_a')

['attr_a', 'attr_b']

### Fetch Data

Session Data

In [297]:
from ai_cdss.services.data import DataLoader
loader = DataLoader(PATIENT_LIST)

In [298]:
session = loader.load_session_data()

Database engine created successfully
Data successfully saved to rgs_interaction.csv
Database engine closed


Deficit Matrix

In [ ]:
import pandas as pd
patient_df = loader.load_patient_data()

In [301]:
from ai_cdss.services.data import ClinicalProcessor

In [302]:
cp = ClinicalProcessor()

In [303]:
deficit = cp.process(patient_df)

Protocol Data

- Fixed Attributes
- Fixed Internal Subscales Names

- Flexible Patients Excel Subscales

In [217]:
protocol_df = loader.load_protocol_data()

Mapping Dict

In [280]:
mkd = MultiKeyDict.from_yaml('mapping.yaml')

Mapping Function

In [294]:
def map_latent_to_clinical(protocol_attributes, mapping_dict, agg_func=np.mean):
    """We need to collapse the protocol feature space into the clinical feature space.
    """
    df_clinical = pd.DataFrame(index=protocol_attributes.index)

    # Collapse using agg_func the protocol latent attributes    
    for clinical_scale, features in mapping_dict.items():
        df_clinical[clinical_scale] = protocol_attributes[features].apply(agg_func, axis=1)

    df_clinical.index = protocol_attributes["PROTOCOL_ID"]

    return df_clinical

### Test modules

Data Modules

In [367]:
from ai_cdss.services.data import DataLoader, ClinicalProcessor, ProtocolProcessor

loader = DataLoader(PATIENT_LIST)

session = loader.load_session_data()
patient = loader.load_patient_data()
protocol = loader.load_protocol_data()

Database engine created successfully
Data successfully saved to rgs_interaction.csv
Database engine closed


In [368]:
# Process Patient - Loads Max Scales
patient_processor = ClinicalProcessor()
deficit = patient_processor.process(patient)

protocol_processor = ProtocolProcessor()
attributes = protocol_processor.process(protocol)

In [369]:
session

,PATIENT_ID,HOSPITAL_ID,PARETIC_SIDE,UPPER_EXTREMITY_TO_TRAIN,HAND_RAISING_CAPACITY,COGNITIVE_FUNCTION_LEVEL,HAS_HEMINEGLIGENCE,GENDER,SKIN_COLOR,AGE,...,PROTOCOL_TYPE,AR_MODE,WEEKDAY_INDEX,REAL_SESSION_DURATION,PRESCRIBED_SESSION_DURATION,SESSION_DURATION,ADHERENCE,TOTAL_SUCCESS,TOTAL_ERRORS,SCORE
0,775,40,LEFT,LEFT,LOW,MEDIUM,0,FEMALE,FDC3AD,88,...,Hands,NONE,4,492,300,300,1.0,99,8,231
1,775,40,LEFT,LEFT,LOW,MEDIUM,0,FEMALE,FDC3AD,88,...,Hands,NONE,4,338,300,300,1.0,64,17,88
2,775,40,LEFT,LEFT,LOW,MEDIUM,0,FEMALE,FDC3AD,88,...,AR,TABLE,4,280,240,240,1.0,0,0,0
3,775,40,LEFT,LEFT,LOW,MEDIUM,0,FEMALE,FDC3AD,88,...,AR,TABLE,4,391,300,300,1.0,1,2,1
4,775,40,LEFT,LEFT,LOW,MEDIUM,0,FEMALE,FDC3AD,88,...,Hands,NONE,0,472,300,300,1.0,86,10,222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2570,3318,41,LEFT,LEFT,MEDIUM,MEDIUM_HIGH,0,MALE,D09C80,75,...,AR,TABLE,3,348,300,300,1.0,43,20,105
2571,3318,41,LEFT,LEFT,MEDIUM,MEDIUM_HIGH,0,MALE,D09C80,75,...,Body,NONE,3,313,300,300,1.0,39,0,39
2572,3318,41,LEFT,LEFT,MEDIUM,MEDIUM_HIGH,0,MALE,D09C80,75,...,AR,TABLE,3,372,300,300,1.0,0,0,288
2573,3318,41,LEFT,LEFT,MEDIUM,MEDIUM_HIGH,0,MALE,D09C80,75,...,AR,TABLE,3,379,300,300,1.0,51,30,84


In [ ]:
def adh(session):
    sult = (
        session.groupby(['PATIENT_ID', 'PROTOCOL_ID'])['ADHERENCE']
        .transform(lambda x: x.ewm(alpha=0.4, adjust=True).mean())
    )
    return result

In [ ]:
def feature_contributions(df_A, df_B):
    # Convert to numpy
    A = df_A.to_numpy()
    B = df_B.to_numpy()

    # Compute row-wise norms
    A_norms = np.linalg.norm(A, axis=1, keepdims=True)
    B_norms = np.linalg.norm(B, axis=1, keepdims=True)
    
    # Replace zero norms with a small value to avoid NaN (division by zero)
    A_norms[A_norms == 0] = 1e-10
    B_norms[B_norms == 0] = 1e-10

    # Normalize each row to unit vectors
    A_norm = A / A_norms
    B_norm = B / B_norms

    # Compute feature contributions
    contributions = A_norm[:, np.newaxis, :] * B_norm[np.newaxis, :, :]

    return contributions

def compute_ppf(patient_data, protocol_data):
    """ Compute the patient-protocol feature matrix (PPF) and feature contributions.
    """
    contributions = feature_contributions(patient_data, protocol_data)
    ppf = np.sum(contributions, axis=2)
    ppf = pd.DataFrame(ppf, index=patient_data.index, columns=protocol_data.index)
    contributions = pd.DataFrame(contributions.tolist(), index=patient_data.index, columns=protocol_data.index)
    
    return ppf, contributions

Scoring Module

In [349]:
index = pd.MultiIndex.from_product([patient.index, protocol.PROTOCOL_ID], names=["PATIENT_ID", "PROTOCOL_ID"])
scoring_df = pd.DataFrame(index=index).reset_index()

In [ ]:
scoring_df.pipe()

,PATIENT_ID,PROTOCOL_ID
0,2110,214
1,2110,223
2,2110,208
3,2110,204
4,2110,205
...,...,...
770,3432,228
771,3432,232
772,3432,230
773,3432,233


In [373]:
ppf, contrib = compute_ppf(patient, attributes)

In [364]:
ppf.reset_index().melt(id_vars=ppf.index.names, var_name="PROTOCOL_ID", value_name="PPF_VALUE")

,PATIENT_ID,PROTOCOL_ID,PPF_VALUE
0,2110,214,0.522497
1,775,214,0.509417
2,788,214,0.453292
3,787,214,0.594709
4,1123,214,0.480499
...,...,...,...
770,2963,229,0.227157
771,3081,229,0.136567
772,3229,229,0.179273
773,3318,229,0.077610


In [ ]:
ppf.stack().reset_index()

,PATIENT_ID,PROTOCOL_ID,0
0,2110,214,0.522497
1,2110,223,0.511424
2,2110,208,0.684176
3,2110,204,0.540294
4,2110,205,0.593878
...,...,...,...
770,3432,228,0.145856
771,3432,232,0.035583
772,3432,230,0.091694
773,3432,233,0.125845
